# <center> **Optimization** </center>
---

The code is structured to take as :

### **Input** : 
- A list of available resources extracted from the file. The resources in the list are in the form of classes : `available_resources`
- 

### **Output** : 
- A list of the best ressources selected for the round : `selected_resources`


In [1]:
import optuna

# Resource class definition
class Resource:
    def __init__(self, name, purchase_cost, maintenance_cost, lifespan, water_consumption, maintenance_frequency, energy_output, side_effects):
        self.name = name  # Resource name
        self.purchase_cost = purchase_cost  # Initial purchase cost
        self.maintenance_cost = maintenance_cost  # Regular maintenance cost
        self.lifespan = lifespan  # Number of operational cycles
        self.water_consumption = water_consumption  # Water required
        self.maintenance_frequency = maintenance_frequency  # Maintenance interventions needed
        self.energy_output = energy_output  # Energy provided by the resource
        self.side_effects = side_effects  # Environmental impact (pollution, noise, etc.)

    def __repr__(self):
        return f"{self.name} (Cost: {self.purchase_cost}, Energy: {self.energy_output})"

# List of available resources
available_resources = [
    Resource("Solar Panel", 2000, 50, 10, 0, 0, 500, 0),
    Resource("Wind Turbine", 5000, 100, 20, 0, 0, 1200, 0),
    Resource("Diesel Generator", 3000, 300, 5, 50, 2, 2000, -100),
    Resource("Battery Storage", 1500, 20, 15, 0, 1, 0, 10),
]

In [7]:
def optimization(available_resources):
    # Optimization function
    def objective(trial):
        selected_resources = []
        budget = 10000  # Maximum budget
        # Iterate through available resources and determine selection
        for resource in available_resources:
            if trial.suggest_categorical(f"use_{resource.name}", [True, False]):
                if budget >= resource.purchase_cost:
                    selected_resources.append(resource)
                    budget -= resource.purchase_cost
        # Compute total energy and total cost
        total_energy = sum(r.energy_output for r in selected_resources)
        total_cost = sum(r.purchase_cost for r in selected_resources) 
        
        if total_cost == 0:
            return float('-inf')  # Penalize cases where no resources are selected
            
        ########################" MOST IMPORTANT PART ###################
        # Calculate score
        score = total_energy / total_cost  # Maximize energy-to-cost ratio
        ########################" MOST IMPORTANT PART ###################
        return score

    # Run optimization with Optuna
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)

    # Display best combination of resources
    print("\nOptimal resource selection:")
    best_params = study.best_trial.params
    for res in available_resources:
        if best_params[f"use_{res.name}"]:
            print(f"- {res.name}")
    
    # Filtrer pour ne garder que les ressources avec 'True'
    selected_resources = [key for key, value in best_params.items() if value]
    return selected_resources


In [8]:
best_ressources = optimization(available_resources)
best_ressources

[I 2025-03-12 17:03:58,482] A new study created in memory with name: no-name-5df1329c-3ab7-4589-b5a6-5a8a47305fca
[I 2025-03-12 17:03:58,487] Trial 0 finished with value: 0.24 and parameters: {'use_Solar Panel': False, 'use_Wind Turbine': True, 'use_Diesel Generator': False, 'use_Battery Storage': False}. Best is trial 0 with value: 0.24.
[I 2025-03-12 17:03:58,491] Trial 1 finished with value: 0.4444444444444444 and parameters: {'use_Solar Panel': False, 'use_Wind Turbine': False, 'use_Diesel Generator': True, 'use_Battery Storage': True}. Best is trial 1 with value: 0.4444444444444444.
[I 2025-03-12 17:03:58,495] Trial 2 finished with value: 0.5 and parameters: {'use_Solar Panel': True, 'use_Wind Turbine': False, 'use_Diesel Generator': True, 'use_Battery Storage': False}. Best is trial 2 with value: 0.5.
[I 2025-03-12 17:03:58,501] Trial 3 finished with value: 0.4444444444444444 and parameters: {'use_Solar Panel': False, 'use_Wind Turbine': False, 'use_Diesel Generator': True, 'use_


Optimal resource selection:
- Diesel Generator


['use_Diesel Generator']